In [ ]:
# default_exp model

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_model.ipynb.
Converted 02_generators.ipynb.
Converted 03_discriminators.ipynb.
Converted 99_diffaugment.ipynb.
Converted Untitled.ipynb.


# Models

> Here be the code for the models.

In [ ]:
#export
import torch
import torch.nn as nn
import torchvision
import defocus.networks.generators as generators
import defocus.networks.discriminators as discriminators
from torch.optim import Adam
from adamp import AdamP

In [ ]:
#export
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.G = None
        self.D = None
        self.P = None
        self.G_opt = None
        self.D_opt = None
        
    def set_generator(self, G):
        # if G is a string, find that model in the generators.py and create it
        if isinstance(G, str):
            self.G = getattr(generators, G)()
        # if not, G is a a network so set it
        else:
            self.G = G
    
    def set_discriminator(self, D):
        # if D is a string, find that model in the discriminators.py and create it
        if isinstance(D, str):
            self.D = getattr(discriminators, D)()
        # if not, D is a a network so set it
        else:
            self.D = D
            
    def set_perceptual(self, after_activation=False):
        # this is from DeblurGANv2
        # TODO: ESRGAN's perceptual loss version
        conv_3_3_layer = 14
        cnn = torchvision.models.vgg19(pretrained=True).features
        perceptual = nn.Sequential()
        perceptual = perceptual.eval()
        for i, layer in enumerate(list(cnn)):
            perceptual.add_module(str(i), layer)
            if i == conv_3_3_layer:
                break
        self.P = perceptual
        
    def set_optimizers(self, optimizer='AdamP', lr=1e-4, betas=(0.9, 0.999), weight_decay=0, nesterov=False):
        # note that lucidrains uses betas=(0.5, 0.9) for stylegan
        # https://github.com/lucidrains/stylegan2-pytorch/blob/master/stylegan2_pytorch/stylegan2_pytorch.py#L565
        
        if optimizer=='Adam':
            self.G_opt = Adam(self.G.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
            self.D_opt = Adam(self.D.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        elif optimizer=='AdamP':
            self.G_opt = AdamP(self.G.parameters(), lr=lr, betas=betas, weight_decay=weight_decay, nesterov=nesterov)
            self.D_opt = AdamP(self.D.parameters(), lr=lr, betas=betas, weight_decay=weight_decay, nesterov=nesterov)
        else:
            #TODO: other optimizers, maybe from the torch_optimizers package
            raise NotImplementedError('nope')
                
    def parallelize(self, device_ids, output_device):
        self.G = nn.parallel.DistributedDataParallel(self.G, device_ids=device_ids, output_device=output_device)
        if self.D is not None:
            self.D = nn.parallel.DistributedDataParallel(self.D, device_ids=device_ids, output_device=output_device)
        if self.P is not None:
            self.P = nn.parallel.DistributedDataParallel(self.P, device_ids=device_ids, output_device=output_device)
            
    def save(self, model_path):
        torch.save({'G':self.G.state_dict(),
                    'D':self.D.state_dict(),
                    'optimizer_G': self.G_opt.state_dict(),
                    'optimizer_D': self.D_opt.state_dict()}, 
                   model_path)
        
    def load(self, model_path, isStrict=False):
        checkpoint = torch.load(model_path)
        self.G.load_state_dict(checkpoint['G'], strict=isStrict)
        self.D.load_state_dict(checkpoint['D'], strict=isStrict)
        self.G_opt.load_state_dict(checkpoint['optimizer_G'], strict=isStrict)
        self.D_opt.load_state_dict(checkpoint['optimizer_D'], strict=isStrict)